In [ ]:
from pathlib import Path
import cv2
import numpy as np

# ---------- Non-white pixel counter ----------
def count_nonwhite_pixels(img: np.ndarray, white_tol: int = 5) -> int:
    """
    Count pixels that are NOT white. A pixel is treated as white if all color
    channels are >= 255 - white_tol. (Default tol=5 handles JPEG noise.)
    Supports BGR (3-ch) and BGRA (4-ch). For BGRA, fully transparent pixels
    are ignored (treated as background/white).
    """
    if img is None:
        raise ValueError("count_nonwhite_pixels: got None image")

    # If alpha present, ignore fully transparent pixels (treat as background)
    if img.ndim == 3 and img.shape[2] == 4:
        b, g, r, a = cv2.split(img)
        is_white_like = (b >= 255 - white_tol) & (g >= 255 - white_tol) & (r >= 255 - white_tol)
        is_visible = (a > 0)
        nonwhite_mask = (~is_white_like) & is_visible
        return int(np.count_nonzero(nonwhite_mask))

    # 3-channel BGR
    if img.ndim == 3 and img.shape[2] == 3:
        b, g, r = cv2.split(img)
        is_white_like = (b >= 255 - white_tol) & (g >= 255 - white_tol) & (r >= 255 - white_tol)
        return int(img.shape[0] * img.shape[1] - np.count_nonzero(is_white_like))

    # Grayscale
    if img.ndim == 2:
        is_white_like = (img >= 255 - white_tol)
        return int(img.size - np.count_nonzero(is_white_like))

    raise ValueError(f"Unsupported image shape: {img.shape}")

# ---------- (Optional) Your existing LAB red mask bits kept here ----------
def red_mask_lab(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    L, A, B = cv2.split(lab)
    # Example thresholds — replace to match your pipeline
    lower_a, upper_a = 135, 140
    lower_l, upper_l = 0, 80
    mask_a = cv2.inRange(A, lower_a, upper_a)
    mask_l = cv2.inRange(L, lower_l, upper_l)
    mask = cv2.bitwise_and(mask_a, mask_l)
    return mask

def count_max_blob_area(img_path: Path) -> int:
    img = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
    if img is None:
        raise FileNotFoundError(f"Could not read: {img_path}")
    mask = red_mask_lab(img)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return int(max((cv2.contourArea(c) for c in contours), default=0))

# ---------- Run on a folder ----------
if __name__ == "__main__":
    test_path = Path("/Users/at/Orbit_Red_Blob/Data/UKE_Plot_14_DD_16_7_2025/sample_noise")
    image_files = sorted([*test_path.glob("*.jpg"), *test_path.glob("*.jpeg"), *test_path.glob("*.png")])

    if not image_files:
        print(f"No images found in {test_path}")
    else:
        for img_path in image_files:
            img = cv2.imread(str(img_path), cv2.IMREAD_UNCHANGED)
            if img is None:
                print(f"SKIP (can't read): {img_path.name}")
                continue

            nonwhite = count_nonwhite_pixels(img, white_tol=5)
            h, w = img.shape[:2]
            pct = 100.0 * nonwhite / (h * w)
            print(f"{img_path.name}: non_white_pixels={nonwhite} ({pct:.2f}% of {w}x{h})")

        # If you also want your old metric:
        for img_path in image_files:
            max_area = count_max_blob_area(img_path)
            print(f"{img_path.name}: max_blob_area={max_area}")
